In [1]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process
import string
import sklearn

import nltk
from nltk import pos_tag, pos_tag_sents
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
nltk.download('stopwords')
stop_words = set(stopwords.words('english')) - set(['at', 'do', 'your', 'from', 'to', 'out', 'no', 'the'])

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/manikya_varshney/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
path = '/home/manikya_varshney/Documents/Python/Yale/All Graphs/Test Data/final_processed_april01.csv'
data = pd.read_csv(path, low_memory=False)

In [3]:
data

,user_id,text_duplicate,final
0,2.843399e+07,As Chicago households fill out 2020 census dur...,chicago household fill out 2020 census coronav...
1,1.010439e+08,Most of the press corps is very busy covering ...,the press corp busy cover the really important...
2,1.647527e+07,@realDonaldTrump Mar 10–he was very specific: ...,mar 10–he specific google 1700 engineer work ...
3,1.647527e+07,"@realDonaldTrump , You were caustic &amp; sarc...",caustic amp sarcastic the rollout come even f...
4,3.453977e+08,"Florida nears 8,000 coronavirus cases, as stat...",florida nears 8000 coronavirus case state repo...
...,...,...,...
16920,9.832288e+17,#SierraLeone has registered its index case of ...,sierraleone register index case covid19 from d...
16921,1.856380e+07,KTAR News reporter @TaylorKinnerup gives a cor...,ktar news reporter give coronavirus update ari...
16922,9.186279e+08,Part of a 1938 public health map: https://t.co...,part 1938 public health map
16923,1.240031e+18,While the resourcefulness and hustle shown by ...,the resourcefulness hustle show many to produc...


In [4]:
keywords = ['stay at home' , 'do your part', 'Responsible', 
            'home', 'house', 'cancel', 'shutdown', 'postpone',
            'school closure', 'Closure', 'business closure',
            'suspension', 'quarantine', 'lockdown', 'social distance', 
            'social distancing', 'self quarantine', 'isolat', '6-feet',
            'distance', '#clubquarantine', '#quarantinelife', '#quarantineacitivites']

##### 1. Casing (Upper or lower case)
##### 2. Noise Removal (Removal of punctuation, white spaces, special characters, HTML tags)
##### 3. Tokenization (Tweets to tokens i.e. words seprated by spaces)
##### 4. Stopword Removal
##### 5. Text Normalization (Stemming and Lemmatization)

In [5]:
#Convert to lower
for i in range(len(keywords)): 
    keywords[i] = keywords[i].lower()

#Remove punctuations   
for i in range(len(keywords)): 
    keywords[i] = keywords[i].translate(str.maketrans('','',string.punctuation))

#More cleaning
for i in range(len(keywords)): 
    keywords[i] = keywords[i].replace('/[^a-zA-Z0-9 ]/g', '').replace('\n',' ').strip('“').strip('“').strip('’').lstrip(' ').rstrip(' ')

#Tokenize
#keywords_tokens = [sub.split() for sub in keywords] 

#Remove stop words
def remove_stopwords(data):
    output_array=[]
    for sentence in data:
        temp_list=[]
        for word in sentence.split():
            if word not in stop_words:
                temp_list.append(word)
        output_array.append(' '.join(temp_list))
    return output_array

keywords_filtered=remove_stopwords(keywords)

#Stemming
ps = PorterStemmer()
keywords_stem = [[ps.stem(word) for word in sentence.split(" ")] for sentence in keywords_filtered]
keywords_stem = [" ".join(sentence) for sentence in keywords_stem]

#Lemmetizing

#POSTags
def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ, "N": wordnet.NOUN, "V": wordnet.VERB, "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = WordNetLemmatizer()
keywords_lem = [[lemmatizer.lemmatize(y, get_wordnet_pos(y)) for y in sentence.split(" ")] for sentence in keywords_filtered]
keywords_final = [" ".join(sentence) for sentence in keywords_lem]

## Fuzzy Matching

In [6]:
data['final'] = data['final'].apply(str)
choices = data['final'].tolist()

In [7]:
def fuzzy_m(row):
        keyword_match, score = process.extractOne(row['final'], keywords_final, scorer = fuzz.partial_ratio)
        row['final_score'] = score
        row['final_keyword_match'] = keyword_match
        return row

In [8]:
nan_value = float("NaN")
data['final'].replace("", nan_value, inplace=True)
data.dropna(subset = ["final"], inplace=True)

In [9]:
interim_imp = data.apply(fuzzy_m, axis=1)

In [10]:
interim_imp

,user_id,text_duplicate,final,final_score,final_keyword_match
0,2.843399e+07,As Chicago households fill out 2020 census dur...,chicago household fill out 2020 census coronav...,100,house
1,1.010439e+08,Most of the press corps is very busy covering ...,the press corp busy cover the really important...,100,home
2,1.647527e+07,@realDonaldTrump Mar 10–he was very specific: ...,mar 10–he specific google 1700 engineer work ...,60,house
3,1.647527e+07,"@realDonaldTrump , You were caustic &amp; sarc...",caustic amp sarcastic the rollout come even f...,75,home
4,3.453977e+08,"Florida nears 8,000 coronavirus cases, as stat...",florida nears 8000 coronavirus case state repo...,53,social distance
...,...,...,...,...,...
16920,9.832288e+17,#SierraLeone has registered its index case of ...,sierraleone register index case covid19 from d...,100,social distance
16921,1.856380e+07,KTAR News reporter @TaylorKinnerup gives a cor...,ktar news reporter give coronavirus update ari...,75,business closure
16922,9.186279e+08,Part of a 1938 public health map: https://t.co...,part 1938 public health map,50,home
16923,1.240031e+18,While the resourcefulness and hustle shown by ...,the resourcefulness hustle show many to produc...,67,isolat


In [11]:
denominator = interim_imp.shape[0]

In [12]:
interim_imp['final_score'] = interim_imp['final_score'].astype(int)

In [13]:
interim_imp = interim_imp[interim_imp['final_score'] == 100]

In [14]:
interim_imp

,user_id,text_duplicate,final,final_score,final_keyword_match
0,2.843399e+07,As Chicago households fill out 2020 census dur...,chicago household fill out 2020 census coronav...,100,house
1,1.010439e+08,Most of the press corps is very busy covering ...,the press corp busy cover the really important...,100,home
18,9.502707e+07,How are we at #jpcarpet spending our #quaranti...,at jpcarpet spending quarantine building close...,100,home
26,2.929292e+07,Happy #NationalPoetryMonth - new pandemic poet...,happy nationalpoetrymonth new pandemic poetry ...,100,quarantine
29,1.245499e+18,I know breakups during this Coronavirus Isolat...,know breakup coronavirus isolation hit hard th...,100,isolat
...,...,...,...,...,...
16901,1.678039e+07,Stay Home. Stay Safe. Save Lives. The state of...,stay home stay safe save life the state michig...,100,home
16910,1.667517e+08,CNN isn’t showing the White House daily briefi...,cnn show the white house daily briefing anymo...,100,house
16917,1.408504e+07,Gov. Ron DeSantis on Tuesday said he had no pl...,gov ron desantis tuesday say no plan to issue ...,100,home
16918,3.690315e+08,We’re here for you. Please #StayHomeWorkSafe f...,please stayhomeworksafe u the late info covid1...,100,home


In [15]:
numerator = interim_imp.shape[0]

In [16]:
proportion = (numerator/denominator)
print(proportion)

0.15397341211225998
